In [ ]:
import cv2
import numpy as np

from EvEye.utils.PupilTracker import PupilTracker
from EvEye.utils.cache.MemmapCacheStructedEvents import *
from EvEye.utils.tonic.functional.ToFrameStack import to_frame_stack_numpy
from EvEye.utils.tonic.slicers.SliceEventsAtIndices import slice_events_at_timepoints
from EvEye.utils.visualization.visualization import *
from EvEye.utils.processor.TxtProcessor import TxtProcessor

In [ ]:
events_path = Path(
    "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeCenterDataset/data/user1_left_session_1_0_1_events.txt"
)
ellipses_path = Path(
    "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeEllipseLabelsV2/user1_left_session_1_0_1.txt"
)
output_path = Path("/mnt/data2T/junyuan/eye-tracking/outputs/PupilTrack")
output_path.mkdir(exist_ok=True, parents=True)
events = TxtProcessor(events_path).load_events_from_txt()
ellipses = TxtProcessor(ellipses_path).load_ellipses_from_txt()
sensor_size = (346, 260, 2)
time_window = 1000
events_interpolation = "causal_linear"
pupil_tracker = PupilTracker()

In [ ]:
index = 31
end_event_time = ellipses[index][0]
start_event_time = max(0, end_event_time - time_window)
event_segment = slice_events_at_timepoints(events, start_event_time, end_event_time)
event_frame_raw = to_frame_stack_numpy(
    event_segment,
    sensor_size,
    1,
    events_interpolation,
    start_event_time,
    end_event_time,
)
event_frame_raw_vis = visualize(event_frame_raw)
ellipse = convert_to_ellipse(ellipses[index])
draw_ellipse(event_frame_raw_vis, ellipse)
save_image(event_frame_raw_vis, f"{output_path}/event_frame_raw.png")
st = start_event_time
et = end_event_time

In [ ]:
ellipse

In [ ]:
for save_index in tqdm(range(100)):
    for track_index in range(100):
        start_event_time = end_event_time
        end_event_time = start_event_time + time_window
        event_segment = slice_events_at_timepoints(
            events, start_event_time, end_event_time
        )
        event_frame = to_frame_stack_numpy(
            event_segment,
            sensor_size,
            1,
            events_interpolation,
            start_event_time,
            end_event_time,
        )
        event_frame_vis = visualize(event_frame)
        pupil_tracker = PupilTracker()
        cv2.ellipse(event_frame_vis, ellipse, (0, 255, 0), 1)
        ellipse = pupil_tracker(event_frame_vis, ellipse)
        cv2.ellipse(event_frame_vis, ellipse, (255, 255, 255), 1)
        save_image(
            event_frame_vis, f"{output_path}/event_frame_{save_index}_{track_index}.png"
        )